### 本Script包含在Bacdive数据集和human gut&sputum数据集上进行引物设计和验证的代码


In [1]:
import os
import re
import sys
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
import math
from collections import Counter

# biopython utils
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO

import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy.stats import wilcoxon
from scipy.stats import chi2_contingency
from matplotlib.ticker import FuncFormatter

# from adjustText import adjust_text
import seaborn as sns

#### 1. 首先处理silva数据库获得用于引物设计所必须的ref文件
Model_data/Silva_ref_data

In [ ]:
## 首先利用16sDeepSeg处理silva数据获得用于引物设计所必须的所有silva seqs的划分结果文件
# 1106: 注意要把silva_id改成id才可以，之前是用silva_id_wrong跑的结果
deepSeg_cmd = 'python 16sDeepSeg/module_output.py -c Model_data/16sDeepSeg_publicated_model/config.json -r Model_data/16sDeepSeg_publicated_model/model.pth --input /data1/hyzhang/Projects/EcoPrimer_git/DeepEcoPrimer_v2/data/SILVA_data/silva_seqs_id_only.csv --output data/SILVA_processed_data/SILVA_set_segmentation_16sDeepSeg.csv'
os.system(deepSeg_cmd)

In [2]:
df_silva = pd.read_csv('data/SILVA_data/silva_seqs_id_only.csv', )
df_silva.head()

,silva_id,silva_id_wrong,16s_rna,lens
0,AB001445.1.1538,AB001445.1.1538,AACTGAAGAGTTTGATCATGGCTCAGATTGAACGCTGGCGGCAGGC...,1538
1,KM209255.204.1909,KM209255.204.1909,AGAGTTTGATCATGGCTCAGATTGAACGCTGGCGGCAGGCCTAACA...,1706
2,HL281554.1.1313,HL281554.1.1313,GACGAACGCTGGCGGCGTGCTTAACACATGCAAGTCGAACGAGTGG...,1313
3,AB002515.1.1332,AB002515.1.1332,GCCTAATACATGCAAGTTGACGACAGATGATACGTAGCTTGCTACA...,1332
4,AB002523.1.1496,AB002523.1.1496,TCCTGGCTCAGGACGAACGCTGGCGGCGTGCCTAATACATGCAAGT...,1496


In [3]:
# makeblastdb: 现在用的是忘了从哪里搞得，比Ecoli K12的NCBI序列更长？后续换一下？
!makeblastdb -in /data1/hyzhang/Projects/16sDeepSeg_summary/Evi_specific_primers_database/Primer_evaluation_fromMetagenomeData/data/Ecoli_ref.fasta -input_type fasta -dbtype nucl -out data/ecoli_db/Ecoli_ref_1550



Building a new DB, current time: 11/06/2023 19:12:28
New DB name:   /data1/hyzhang/Projects/EcoPrimer_git/DeepEcoPrimer_v2/data/ecoli_db/Ecoli_ref_1550
New DB title:  /data1/hyzhang/Projects/16sDeepSeg_summary/Evi_specific_primers_database/Primer_evaluation_fromMetagenomeData/data/Ecoli_ref.fasta
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 1 sequences in 0.005512 seconds.


In [4]:
## 计算所有silva seqs中包含了完整的ecoli 8bp-1492bp的序列
## 这个script用来构建包含全长序列的silva数据库，从431388的全长序列出发，就跟mothur无关了
def blast_cmd(query_primer="", db="", out_path = ''):
    cmd = f"blastn -db {db} -query "
    cmd += query_primer
    cmd += " -outfmt 7 "
    cmd += "-out "
    cmd += out_path
    cmd += " -task blastn-short -word_size 4 -evalue 1 -max_target_seqs 1000000"
    
    os.system(cmd)

# primer blast results
def parse_blastTxt(blast_txt = '/data3/hyzhang/ont/16s_RNA_seg/res/blast_res/query.txt'):
    with open(blast_txt, 'r') as f:
        lines = f.readlines()
    index_line = lines[3]
    index_line = index_line.split(': ')[-1]
    index_line = index_line.split(',')
    index_line = [x.strip(' ') for x in index_line]
    index_line = ['_'.join(x.split()) for x in index_line]
    print(index_line)

    lines = [x.strip('\n').split('\t') for x in lines if not x.startswith('#')]
    
    blast_df = pd.DataFrame(lines)
    blast_df.columns = index_line
    
    # os.remove(blast_txt)
    return blast_df


# 以下是序列比对和操作过程
ecoli_fromPos_cut = 8
ecoli_endPos_cut = 1500

def filter_seqs(silva_ori_file = 'data/SILVA_data/silva_seqs_id_only.csv'):
    # read csv info file containing all the taxa and seq ids.
    silva_database_seqs = pd.read_csv(silva_ori_file)
    # write a fasta file
    records = []
    for i in range(silva_database_seqs.shape[0]):
        seq_now = silva_database_seqs['16s_rna'][i]
        seq_now = Seq(seq_now)
        id_now = silva_database_seqs['silva_id'][i]
        
        rec = SeqRecord(seq=seq_now, id=id_now, description= '')
        records.append(rec)

    with open('data/SILVA_processed_data/silva_seqs_all_1106.fasta', 'w') as f:
        SeqIO.write(records, f, 'fasta')
        
    # blast every seqs to Ecoli ref seq
    silva_seqs = 'data/SILVA_processed_data/silva_seqs_all_1106.fasta'
    out_path = silva_seqs.replace(".fasta", "_alignToEcoli.fasta")
    db = 'data/ecoli_db/Ecoli_ref_1550'

    blast_cmd(query_primer=silva_seqs, out_path=out_path, db=db)

    # parse res of the blast
    from tqdm import tqdm
    blast_df = parse_blastTxt(out_path)
    need_to_be_int = ['s._start', 's._end', 'q._start', 'q._end', 'alignment_length']
    for ind in need_to_be_int:
        blast_df[ind] = blast_df[ind].apply(int)
    blast_df['evalue'] = blast_df['evalue'].apply(float)
    blast_res_df = []
    for seq_id, seq_df in tqdm(blast_df.groupby('query_acc.ver')):
        res_t = {'seq_id':seq_id}
        seq_df.sort_values(by='s._start', ascending=True, inplace=True)
        seq_df = seq_df[(seq_df['alignment_length'] >= 15) & (seq_df['evalue'] < 0.5)]
        seq_df.reset_index(inplace=True, drop=True)
        
        silva_info = silva_database_seqs[silva_database_seqs['silva_id'] == seq_id]
        seq_lens = int(silva_info.reset_index()['lens'][0]) # full lens including the 8f and 1492r region
        
        # get info 
        qs1, qe1, ss1, se1 = list(seq_df.loc[0, ['q._start', 'q._end', 's._start', 's._end']]) 
        qs2, qe2, ss2, se2 = list(seq_df.loc[seq_df.shape[0] - 1, ['q._start', 'q._end', 's._start', 's._end']])
        
        start_bp_correspondingSiteInEcoli = ss1 - qs1
        end_bp_correspondingSiteInEcoli = seq_lens - qe2 + se2
        res_t['start_bp_correspondingSiteInEcoli'] = start_bp_correspondingSiteInEcoli
        res_t['end_bp_correspondingSiteInEcoli'] = end_bp_correspondingSiteInEcoli
        blast_res_df.append(res_t)
        
    blast_res_df = pd.DataFrame(blast_res_df)
    
    # chose the cut-off value.
    silva_with_complete_8f_and_1492r = blast_res_df[(blast_res_df['start_bp_correspondingSiteInEcoli'] <= ecoli_fromPos_cut) & (blast_res_df['end_bp_correspondingSiteInEcoli'] >= ecoli_endPos_cut)]
    silva_with_complete_8f_and_1492r.reset_index(inplace=True, drop=True)
    silva_with_complete_8f_and_1492r.to_csv(f'data/SILVA_processed_data/silva_with_complete_{ecoli_fromPos_cut}f_{ecoli_endPos_cut}r.csv', index=False)

def merge_with_metadata(filter_csv = f'data/SILVA_processed_data/silva_with_complete_{ecoli_fromPos_cut}f_{ecoli_endPos_cut}r.csv'):
    ori_silva_meta = pd.read_csv('data/SILVA_data/silva_metainfo_clear.csv')
    ori_silva_idSeqs = pd.read_csv('data/SILVA_data/silva_seqs_id_only.csv')
    filtered_seqs = pd.read_csv(filter_csv)
    # 按照silva id 进行merge，然后保留wrong id进行后续primer design
    filter_silva_meta = ori_silva_meta[ori_silva_meta['silva_id'].isin(list(filtered_seqs['seq_id']))]
    filter_silva_idSeqs = ori_silva_idSeqs[ori_silva_idSeqs['silva_id'].isin(list(filtered_seqs['seq_id']))]
    filter_silva_meta.reset_index(inplace = True, drop = True)
    filter_silva_idSeqs.reset_index(inplace = True, drop = True)
    
    filter_silva_meta.to_csv(f'data/SILVA_processed_data/silva_with_complete_{ecoli_fromPos_cut}f_{ecoli_endPos_cut}r_TaxaMeta.csv', index=False)
    filter_silva_idSeqs.to_csv(f'data/SILVA_processed_data/silva_with_complete_{ecoli_fromPos_cut}f_{ecoli_endPos_cut}r_IdSeqs.csv', index=False)
    
    
if __name__ == '__main__':
    filter_seqs()
    merge_with_metadata()


['query_acc.ver', 'subject_acc.ver', '%_identity', 'alignment_length', 'mismatches', 'gap_opens', 'q._start', 'q._end', 's._start', 's._end', 'evalue', 'bit_score']


100%|██████████| 431575/431575 [7:32:48<00:00, 15.89it/s]   


In [3]:
## 生成后续用于in-silico验证的关键文件

# 1106: 拼接一下目前筛选出来的14w的seqs们
idseq_df = pd.read_csv(f'data/SILVA_processed_data/silva_with_complete_{ecoli_fromPos_cut}f_{ecoli_endPos_cut}r_IdSeqs.csv')
idseq_df.drop(columns=['lens'], axis=1, inplace=True)
meta_df = pd.read_csv(f'data/SILVA_processed_data/silva_with_complete_{ecoli_fromPos_cut}f_{ecoli_endPos_cut}r_TaxaMeta.csv')

id_seq_taxa_df = pd.merge(idseq_df, meta_df, on=['silva_id', 'silva_id_wrong'], how='inner')
filter_seqs_df = id_seq_taxa_df

# write the fasta file to the corresponding fasta.
silva_nr_filtered_fasta = []
for seq, id_, des in zip(list(filter_seqs_df['16s_rna']), list(filter_seqs_df['silva_id']), list(filter_seqs_df['taxa_info_summary'])):
    seq_now = Seq(seq)
    rec = SeqRecord(seq=seq_now, id=id_, description=des)
    
    if 'Mitochondria' in des:
        continue
    silva_nr_filtered_fasta.append(rec)
# SeqIO.write(silva_nr_filtered_fasta, 'data/SILVA_processed_data/InsilicoPCR_for_PrimerMatch/silva_seqs_157515_10f_1500r.fasta', 'fasta')


# write the metainfo containing taxa_info and rna seqs
filter_seqs_notMito_df = filter_seqs_df[filter_seqs_df['taxa_info_summary'].apply(lambda x: 'Mitochondria' not in x)]
filter_seqs_notMito_df.reset_index(inplace=True, drop=True)
# filter_seqs_notMito_df = pd.DataFrame(filter_seqs_notMito_df, columns=['silva_id_ori_fromSilva', 'taxa_info', '16s_rna'])

filter_seqs_notMito_df = filter_seqs_notMito_df[filter_seqs_notMito_df['Genus'].apply(lambda x: len(x) > 1)]
filter_seqs_notMito_df.reset_index(inplace = True, drop = True)
print(filter_seqs_notMito_df.shape[0])
filter_seqs_notMito_df.to_csv('data/SILVA_processed_data/InsilicoPCR_for_PrimerMatch/silva_metaInfo_157515_10f_1500r.csv', index=False)


156813


,silva_id,silva_id_wrong,16s_rna,taxa_info_summary,ori_rank_tree_num,Kingdom,Phylum,Class,Order,Family,Genus,Species
0,AB001445.1.1538,AB001445.1.1538,AACTGAAGAGTTTGATCATGGCTCAGATTGAACGCTGGCGGCAGGC...,Bacteria;Proteobacteria;Gammaproteobacteria;Ps...,7,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Pseudomonadaceae,Pseudomonas,Pseudomonas amygdali pv. morsprunorum
1,KM209255.204.1909,KM209255.204.1909,AGAGTTTGATCATGGCTCAGATTGAACGCTGGCGGCAGGCCTAACA...,Bacteria;Proteobacteria;Gammaproteobacteria;En...,7,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Pectobacteriaceae,Dickeya,Dickeya phage phiDP10.3
2,JN088732.1.1500,JN088732.1.1500,AGAGTTTGATCMTGGCTCAGATTGAACGCTGGCGGCAGGCCTAACA...,Bacteria;Proteobacteria;Gammaproteobacteria;Ps...,7,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Porticoccaceae,Porticoccus,Porticoccus hydrocarbonoclasticus
3,JN167515.1.1446,JN167515.1.1446,AGAGTTTGATCCTGGCTCAGAACGAACGCTGGCGGCAGGCCTAACA...,Bacteria;Proteobacteria;Alphaproteobacteria;Rh...,7,Bacteria,Proteobacteria,Alphaproteobacteria,Rhizobiales,Stappiaceae,Pseudovibrio,Pseudovibrio axinellae
4,HG726039.1.1478,HG726039.1.1478,AGAGTTTGATCCTGGCTCAGGACGAACGCTGGCGGCGTGCCTAACA...,Bacteria;Firmicutes;Clostridia;Clostridiales;C...,7,Bacteria,Firmicutes,Clostridia,Clostridiales,Clostridiaceae,Clostridium sensu stricto 1,Clostridium saudiense
...,...,...,...,...,...,...,...,...,...,...,...,...
156808,CFFF01000203.442.2194,CFFF01000203.442.2194,ACAGGCTTATCTCCCCCAAGAGTTCACATCGACGGGGAGGTTTGGC...,Bacteria;Firmicutes;Bacilli;Lactobacillales;St...,7,Bacteria,Firmicutes,Bacilli,Lactobacillales,Streptococcaceae,Streptococcus,Streptococcus pneumoniae
156809,CXYW01000009.360.1734,CXYW01000009.360.1734,GAGAGTTTGATCCTGGCTCAGGATGAACGCTGGCGGCGTGCTTAAC...,Bacteria;Firmicutes;Clostridia;Lachnospirales;...,7,Bacteria,Firmicutes,Clostridia,Lachnospirales,Lachnospiraceae,Epulopiscium,Lachnospiraceae bacterium mt14
156810,CXWL01024289.1436.2731,CXWL01024289.1436.2731,AGAGTTTGATCCTAGCTCAGGGTGAATGCTGGCGGCGTGGATAAGT...,Bacteria;Patescibacteria;Parcubacteria;Candida...,5,Bacteria,Patescibacteria,Parcubacteria,Candidatus Kaiserbacteria,groundwater metagenome,unknown,unknown
156811,MVBC01000034.3.1520,MVBC01000034.3.1520,TTACAATGGAGAGTTTGATCCTGGCTCAGGATGAACGCTAGCGGCA...,Bacteria;Bacteroidota;Bacteroidia;Cytophagales...,7,Bacteria,Bacteroidota,Bacteroidia,Cytophagales,Hymenobacteraceae,Hymenobacter,Hymenobacter sp. CRA2


#### 2. Bacdive数据库中多种类型环境的in-silico验证结果

首先清洗bacdive数据库

In [8]:
## 首先bacdive数据库中的genus id和silva id对应
# original data and data root
bacdive_ori_data = 'data/Bacdive/isolation_sources_bacdive_2022-10-07.csv'
bacdive_root = 'data/Bacdive'

# create a cross table
cols = ['ID', 'Species', 'Isolation source', 'Country', 'Continent', 'Category 3']
bacdive_ori_df = pd.read_csv(bacdive_ori_data)
bacdive_ori_df = pd.DataFrame(bacdive_ori_df, columns=cols)
print('Original dataset shape:', bacdive_ori_df.shape)

# fill id and species na
bacdive_fillID_df = bacdive_ori_df.copy()
bacdive_fillID_df['ID'] = bacdive_fillID_df['ID'].fillna(method='ffill')
bacdive_fillID_df['Species'] = bacdive_fillID_df['Species'].fillna(method='ffill')

# create multi-label dataframe
bacdive_env_labels_Species_df = pd.crosstab(bacdive_fillID_df['Species'], bacdive_fillID_df['Category 3'])
bacdive_env_labels_Species_df = pd.DataFrame(bacdive_env_labels_Species_df)
bacdive_env_labels_Species_df.columns = list(bacdive_env_labels_Species_df.columns)
bacdive_env_labels_Species_df.to_csv(os.path.join(bacdive_root, 'bacdive_micro_Species_labels_1030.tsv'), sep = '\t')

# convert the species name to species ID
# !Rscript convert_taxid_Bacdive.r # output file bacdive_micro_Species_labels_taxid.tsv

# get all the species id for bacdive taxa
bacdive_id = 'data/Bacdive/bacdive_micro_Species_labels_taxid.tsv'
bacdive_taxid = pd.read_csv(bacdive_id)
bacdive_taxid = list(bacdive_taxid['Species'])
bacdive_taxid = [str(int(x)) if (not math.isnan(x)) else '-1' for x in bacdive_taxid]
bacdive_taxid = list(set(bacdive_taxid))

bacdive_taxid = {'bacdive_taxid': bacdive_taxid}
bacdive_taxid = pd.DataFrame(bacdive_taxid)
bacdive_taxid.to_csv(f'{bacdive_root}/bacdive_all_species_taxid.csv', index=False)


Original dataset shape: (126545, 6)


使用外源的R代码根据species id获取其各层级的物种id，bacdive_all_taxa.csv

In [5]:
# 有了NCBI获取的species id之后合并table，保存bacdive所有species的taxonomic metainfo
bacdive_spe = pd.read_csv(f'{bacdive_root}/bacdive_micro_Species_labels.tsv', sep='\t')
bacdive_spe = list(bacdive_spe['Species'])

bacdive_taxid = pd.read_csv(f'{bacdive_root}/bacdive_micro_Species_labels_taxid.tsv')
bacdive_taxid = list(bacdive_taxid['Species'])
bacdive_taxid = [str(int(x)) if (not math.isnan(x)) else '-1' for x in bacdive_taxid]

bacdive_metainfo = {'bacdive_spe_nm': bacdive_spe, 'taxaid': bacdive_taxid}
bacdive_metainfo = pd.DataFrame(bacdive_metainfo)

jinyuan_parsed_speids = f'{bacdive_root}/bacdive_all_taxa.csv'
jinyuan_parsed_df = pd.read_csv(jinyuan_parsed_speids)
jinyuan_parsed_df['taxaid'] = jinyuan_parsed_df['taxaid'].apply(str)

bacdive_metainfo = pd.merge(bacdive_metainfo, jinyuan_parsed_df, on = 'taxaid', how = 'left')
for col in ['taxaid', 'genus_id', 'species_id']:
    bacdive_metainfo[col].fillna(-1, inplace=True)
    bacdive_metainfo[col] = bacdive_metainfo[col].apply(lambda x: str(int(x)))
bacdive_metainfo.to_csv(f'{bacdive_root}/bacdive_metainfo.csv', index=False)

# read bacdive_metainfo.csv
bacdive_metainfo = pd.read_csv(f'{bacdive_root}/bacdive_metainfo.csv')
bacdive_metainfo_only_genus = pd.DataFrame(bacdive_metainfo, columns=['bacdive_spe_nm', 'genus_id', 'genus_name', 'kingdom_name'])
bacdive_metainfo_only_genus.head()

cols = ['ID', 'Species', 'Isolation source', 'Country', 'Continent', 'Category 3']
bacdive_ori_df = pd.read_csv(bacdive_ori_data)
bacdive_ori_df = pd.DataFrame(bacdive_ori_df, columns=cols)
print('Original dataset shape:', bacdive_ori_df.shape)

# fill id and species na
bacdive_fillID_df = bacdive_ori_df.copy()
bacdive_fillID_df['ID'] = bacdive_fillID_df['ID'].fillna(method='ffill')
bacdive_fillID_df['Species'] = bacdive_fillID_df['Species'].fillna(method='ffill')

# simple way to translate species to genus
bacdive_fillID_df = pd.merge(bacdive_fillID_df, bacdive_metainfo_only_genus, left_on='Species', right_on='bacdive_spe_nm', how='left')
bacdive_fillID_df = bacdive_fillID_df[bacdive_fillID_df['kingdom_name'] == 'Bacteria']
print('Only bacteria shape:', bacdive_fillID_df.shape)
bacdive_fillID_df = bacdive_fillID_df[(~bacdive_fillID_df['genus_name'].isnull()) & (bacdive_fillID_df['genus_name'] != '-1')]
print('With genus name shape:', bacdive_fillID_df.shape)
bacdive_fillID_df.reset_index(inplace=True,drop=True)
print(bacdive_fillID_df.head())

# to genus level
# create multi-label dataframe
bacdive_env_labels_Species_df = pd.crosstab(bacdive_fillID_df['genus_name'], bacdive_fillID_df['Category 3'])
bacdive_env_labels_Species_df = pd.DataFrame(bacdive_env_labels_Species_df)
bacdive_env_labels_Species_df.columns = list(bacdive_env_labels_Species_df.columns)

# create environment info table
bacdive_env_labels_Species_df[bacdive_env_labels_Species_df >= 1] = 1
bacdive_env_micro_spe = bacdive_env_labels_Species_df.sum(axis=0).sort_values(ascending=False)

bacdive_env_micro_df = pd.DataFrame({'Genus_num': bacdive_env_micro_spe})
bacdive_env_micro_df['Information'] = [''] * bacdive_env_micro_df.shape[0]
bacdive_env_micro_df['Micro_spe'] = [''] * bacdive_env_micro_df.shape[0]
envs = list(bacdive_env_micro_df.index)

for env in envs:
    bacdive_env_labels_Species_df_t = bacdive_env_labels_Species_df[bacdive_env_labels_Species_df[env] >= 1]
    env_micro_ls = list(bacdive_env_labels_Species_df_t.index)
    bacdive_env_micro_df.loc[env, 'Micro_spe'] = str(env_micro_ls)
    
    if len(env_micro_ls) != bacdive_env_micro_df.loc[env, 'Genus_num']:
        print('aa')

bacdive_env_micro_df.to_csv(os.path.join(bacdive_root, 'Env_micro_list_genus.tsv'), sep = '\t')


Original dataset shape: (126545, 6)
Only bacteria shape: (118002, 10)
With genus name shape: (117680, 10)
         ID                      Species  \
0  159652.0  Abditibacterium utsteinense   
1  159652.0  Abditibacterium utsteinense   
2     219.0        Abiotrophia defectiva   
3     219.0        Abiotrophia defectiva   
4     219.0        Abiotrophia defectiva   

                                    Isolation source     Country  \
0  Top surface sample consisting of weathered gra...  Antarctica   
1                                                NaN         NaN   
2            blood of bacterial endocarditis patient      France   
3                                                NaN         NaN   
4                                                NaN         NaN   

               Continent Category 3               bacdive_spe_nm   genus_id  \
0  Australia and Oceania  #Geologic  Abditibacterium utsteinense  2109263.0   
1                    NaN    #Alpine  Abditibacterium utsteinen

对bacdive所有环境设计引物&in-silico PCR验证&挑选最优引物输出

In [3]:
vv_type_df = pd.read_csv('data/vvs_ontarget_marker.csv', index_col=0)
vv_type_df['vv'] = list(vv_type_df.index)
vv_type_df = pd.DataFrame(vv_type_df, columns=['vv', 'marker'])
mid_size_vvs = list(vv_type_df[vv_type_df['marker'] == 'M']['vv'])

# 设计引物用的参数
mid_size_vvs = ';'.join(mid_size_vvs) # 输入的candidate vv区域
num_every_spe = 50
extend_bp_num = 50
primer_lens = '20,20'
k_num=3

pcr_root = os.path.join(bacdive_root, 'PCR_validation')
bacdive_pri_root = os.path.join(bacdive_root, 'Designed_primers')
os.makedirs(bacdive_pri_root, exist_ok=True)
    
# read bacdive tsv file
bacdive_csv = os.path.join(bacdive_root, 'Env_micro_list_genus.tsv')
environment_info_df = pd.read_csv(bacdive_csv, sep='\t', index_col=0)
environment_info_df['Micro_spe'] = environment_info_df['Micro_spe'].apply(eval)
environment_info_df['Environment_name'] = list(environment_info_df.index)

# simple filter
environment_info_df = environment_info_df[(environment_info_df['Genus_num'] >= 1)]
environment_info_df.reset_index(inplace=True, drop=True)

envi_primers_path = []
design_failed_num = 0
    
# start design
def create_abunTab(core_microbiota = [], file_to = ''):
    df_abun = {'Genus': core_microbiota, 'err_a': len(core_microbiota) * [0.1], 'err_b': len(core_microbiota) * [0.1]}
    df_abun = pd.DataFrame(df_abun)
    df_abun.to_csv(file_to, index=False)
    
for i in tqdm(range(environment_info_df.shape[0])):
    info_t = {}
    microbiota_target = environment_info_df['Environment_name'][i]
    info_t['ori_name'] = microbiota_target
    microbiota_target = microbiota_target.strip().strip('#')
    microbiota_target = '_'.join(microbiota_target.split())
    microbiota_target = microbiota_target.replace('/', '_or_')
    microbiota_target = microbiota_target.replace('(', '').replace(')', '') # special character that cannot be handled
    
    core_microbiota = environment_info_df['Micro_spe'][i]
    out_dir = os.path.join(bacdive_pri_root, microbiota_target)
    input_csv = f'{out_dir}/abunTab.csv'
    os.makedirs(out_dir, exist_ok=True)
    create_abunTab(core_microbiota, f'{out_dir}/abunTab.csv')
    
    # to run DEcoPrimer
    bash_file = f'{out_dir}/run_DEcoPrimer.sh'
    with open(bash_file, 'w') as f:
        try:
            cmd1 = f"python DEcoPrimer.py --input {input_csv} --out_root {out_dir} --input_type 'genera_profiling' --target_vs '{mid_size_vvs}' --extend_bp_num {extend_bp_num} --num_every_spe {num_every_spe} --NGS_mode Single_end"
            os.system(cmd1)
            cmd2 = f"python Insilico_eva_primers.py --envi_forEva {out_dir} --primers_forEva '{out_dir};' --target_vs '{mid_size_vvs}' --K {k_num} --output {pcr_root}/{microbiota_target}"
            os.system(cmd2)
            cmd3 = f"python Screen_best_PP.py --pcr_dir {pcr_root}/{microbiota_target} --rk_by Genus_accuracy"
            os.system(cmd3)
        except:
            design_failed_num += 1
        f.write(cmd1, cmd2, cmd3)
        
    
    # record some metainfo
    info_t['clear_name'] = microbiota_target
    envi_primers_path.append(info_t)


print(f'Env design failed: {design_failed_num}.')

  0%|          | 0/285 [00:00<?, ?it/s]

Total set has 157404 seqs.
Environment has 815 genus.
Target v1v3


  0%|          | 0/9 [00:00<?, ?it/s]

Maybe unusual genus name: candidatus protochlamydia.
Maybe unusual genus name: candidatus solibacter.
Maybe unusual genus name: halalkalibacterium (ex joshi et al. 2022).
Have -1. 1 -1
Get 13510 records.



muscle 5.1.linux64 [12f0e2]  1056Gb RAM, 192 cores
Built Jan 13 2022 23:17:13
(C) Copyright 2004-2021 Robert C. Edgar.
https://drive5.com

Input: 13510 seqs, length avg 99 max 99

00:00 10Mb    100.0% Derep 8131 uniques, 5378 dupes
00:00 12Mb   CPU has 192 cores, running 32 threads                   
01:44 15Mb    100.0% UCLUST 8131 seqs EE<0.01, 3498 centroids, 4632 members
03:06 2.4Gb   100.0% UCLUST 3498 seqs EE<0.30, 125 centroids, 3372 members 
03:06 2.4Gb   100.0% Make cluster MFAs                                    
125 clusters pass 1                   
03:27 2.4Gb   100.0% UCLUST 781 seqs EE<0.10, 101 centroids, 679 members
03:27 2.4Gb   100.0% Make cluster MFAs                                  
03:51 2.4Gb   100.0% UCLUST 761 seqs EE<0.10, 124 centroids, 636 members
03:51 2.4Gb   100.0% Make cluster MFAs                                  
348 clusters pass 2                   
03:51 2.4Gb  
03:51 2.4Gb  Align cluster 1 / 348 (61 seqs)
03:51 2.4Gb  
03:57 2.4Gb   100.0% Calc p

All position after alignment: (961, 8)
After drop - positon: (135, 8)


rm: cannot remove 'data/Bacdive/Designed_primers/Soil/designed_primers/v1v3/forward_pri_cov_temp.txt': No such file or directory
primer_match: fasta_io.t:733: void StreamedFastaFile<T, H>::pos(FILE_POSITION_TYPE) [with T = MapFileChars; H = Lazy_Header_SI; FILE_POSITION_TYPE = long int]: Assertion `0' failed.
Aborted (core dumped)


Maybe unusual genus name: candidatus protochlamydia.
Maybe unusual genus name: candidatus solibacter.
Maybe unusual genus name: halalkalibacterium (ex joshi et al. 2022).
Have -1. 1 -1
Get 13496 records.



muscle 5.1.linux64 [12f0e2]  1056Gb RAM, 192 cores
Built Jan 13 2022 23:17:13
(C) Copyright 2004-2021 Robert C. Edgar.
https://drive5.com

Input: 13496 seqs, length avg 128 max 134

00:00 12Mb    100.0% Derep 6507 uniques, 6988 dupes
00:00 14Mb   CPU has 192 cores, running 32 threads                   


对bacdive结果分析并生成文章结果图

In [1]:
## 获取每个环境的设计结果+整理metainfo+预处理数据
bacdive_ori_df = pd.read_csv('data/Bacdive/isolation_sources_bacdive_2022-10-07.csv')
bacdive_ori_df = bacdive_ori_df.dropna(subset=['Category 3']).reset_index(drop=True)
bacdive_ori_df = pd.DataFrame(bacdive_ori_df, columns=['Category 1', 'Category 2', 'Category 3'])
bacdive_ori_df.drop_duplicates(inplace=True)
bacdive_ori_df.reset_index(drop=True,inplace=True)

bacdive_to_df = pd.read_csv('data/Bacdive/Results/Env_micro_bacdive.csv')
bacdive_to_df = pd.merge(bacdive_to_df, bacdive_ori_df, how='left', left_on='Environment_name', right_on='Category 3').drop(columns=['Category 3'])
print(bacdive_to_df['Category 2'].value_counts())
bacdive_to_df.to_csv('data/Bacdive/Results/Env_micro_bacdive_mergeInfo.csv', index=False) # 保存metainfo table

# 获取引物设计结果路径和信息
bacdive_root = 'data/Bacdive'
evi_list = os.listdir(bacdive_root)
evi_list = [x for x in evi_list if os.path.isdir(os.path.join(bacdive_root, x))]
rank_by = 'Genus_accuracy' # 'pri_amp_eff' 'Genus_accuracy'
vvs_midlen = ['v1v2', 'v1v3', 'v4v5', 'v3v4', 'v6v8', 'v5v7', 'v7v8', 'v4v4', 'v5v6']

univer_primer_table = pd.read_excel('data/Bacdive/Results/primers_sequencing_bacdiveComp.xlsx')# 进行细致比较的时候会用到
univer_primer_table = {x: f'{y}_{x}' for x, y in zip(univer_primer_table['v_region'], univer_primer_table['pri_nm'])}

len(evi_list), univer_primer_table

NameError: name 'pd' is not defined

In [ ]:
# 1. 做的genus-level的均值的结果
bacdive_metainfo = pd.read_csv(f'{bacdive_root}/Env_micro_bacdive_mergeInfo.csv')
bacdive_metainfo.drop(columns=['Information', 'Micro_spe'], inplace=True)

rk_by = 'Genus_accuracy'
df_res_all_evi = pd.read_csv(f'{bacdive_root}/bacdive_designed_res_{rk_by}.csv')

def get_vv_region(pri_str, ):
    pt = 'v\dv\d'
    vv_region = re.findall(pt, pri_str)[0]
    if pri_str.endswith(vv_region):
        pri_type = 'universal'
    else:
        pri_type = 'designed'
    return vv_region, pri_type

df_res_all_evi['VV_region'] = df_res_all_evi['Primer'].apply(lambda x:get_vv_region(x)[0])
df_res_all_evi['Pri_type'] = df_res_all_evi['Primer'].apply(lambda x:get_vv_region(x)[1])
df_res_all_evi = df_res_all_evi[df_res_all_evi['VV_region'].isin(vvs_midlen)] # 保证只在这4个v区域中间选择
df_res_all_evi.reset_index(inplace=True, drop=True)

# 找每个evi的结果
bacdive_best_primer = []
for evi, df in df_res_all_evi.groupby('Environment'):
    idx = df['Value'].idxmax()
    uni_pri_res = {}
    for uni_reg, uni_nm in univer_primer_table.items():
        try:
            v3v4_uni_acc = list(df[df['Primer'] == uni_nm]['Value'])[0]
        except:
            v3v4_uni_acc = 0.0
        uni_pri_res[uni_nm] = v3v4_uni_acc
    
    info_t = {'Environment': evi, 'Primer': df.loc[idx, 'Primer'], 'Value': df.loc[idx, 'Value'], 'VV_region': df.loc[idx, 'VV_region'], 'Pri_type': df.loc[idx, 'Pri_type']}
    info_t.update(uni_pri_res)
    
    # 看一下设计引物的最优
    df_de = df[df['Pri_type'] == 'designed'].copy()
    df_de.reset_index(inplace=True, drop=True)
    idx = df_de['Value'].idxmax()
    info_t.update({'Designed_best_val': df_de.loc[idx, 'Value'], 'Designed_best_primer': df_de.loc[idx, 'Primer'], 'Designed_best_vv_region': df_de.loc[idx, 'VV_region']})
    
    # 看一下通用引物的最优
    df_un = df[df['Pri_type'] == 'universal'].copy()
    df_un.reset_index(inplace=True, drop=True)
    idx = df_un['Value'].idxmax()
    info_t.update({'Universal_best_val': df_un.loc[idx, 'Value'], 'Universal_best_primer': df_un.loc[idx, 'Primer'], 'Universal_best_vv_region': df_un.loc[idx, 'VV_region']})
    
    bacdive_best_primer.append(info_t)
    
bacdive_best_primer = pd.DataFrame(bacdive_best_primer)
bacdive_best_primer = pd.merge(bacdive_best_primer, bacdive_metainfo, how='left', left_on='Environment', right_on='Environment_name_processed')
# print(bacdive_best_primer[bacdive_best_primer['Environment'].duplicated()])
bacdive_best_primer['De_minus_bestUni'] = bacdive_best_primer['Designed_best_val'] - bacdive_best_primer['Universal_best_val']
bacdive_best_primer.to_csv(f'results/bacdive/bacdive_best_primer_{rk_by}.csv', index=False)

# 1. 画一个柱状图
bacdive_best_primer = bacdive_best_primer[bacdive_best_primer['Genus_num'] > 10] # 按照target genus数目做一下筛选
print(f'Total envi number: {bacdive_best_primer.shape}')
bacdive_best_primer.reset_index(inplace=True, drop=True)
bacdive_best_primer['bar_label'] = bacdive_best_primer['VV_region'] + '_' + bacdive_best_primer['Pri_type']
bar_plot_info_t = dict(bacdive_best_primer['bar_label'].value_counts())
bar_plot_info = {k: v for k, v in bar_plot_info_t.items() if 'desi' in k}
bar_plot_info.update({k: v for k, v in bar_plot_info_t.items() if 'uni' in k})

# plt.style.use('ggplot')
plt.figure(figsize=(8, 15))
fig, ax = plt.subplots()

x, y = list(bar_plot_info.keys()), list(bar_plot_info.values())
x_val = list(range(len(y))) 
x_des_only = x_val[: -2] # 因为de包含所有的v区域，所以要去掉最后两个

colors = {'Universal': '#8ECFC9', 'Designed': '#FA7F6F'}
uni_des_gps = {'Universal':{'x': [], 'y': []}, 'Designed':{'x': [], 'y': []}}

# 先放designed，再放universal
x_for_uni = {'v4v5_universal': 1, 'v3v4_universal': 0}
for i, label in enumerate(x):
    if 'designed' in label:
        uni_des_gps['Designed']['x'].append(x_val[i])
        uni_des_gps['Designed']['y'].append(y[i])
    else:
        uni_des_gps['Universal']['x'].append(x_for_uni[label])   # x_val[i])
        uni_des_gps['Universal']['y'].append(y[i])

wid = 0.3
for lb, idx in uni_des_gps.items():
    if lb == 'Universal':
        plt.bar(np.array(idx['x']) + wid / 2, np.array(idx['y']), width=wid, label=lb + ' primer pairs', color=colors[lb], )
        for i_x in x_des_only:
            try:
                x_, y_ = idx['x'][i_x], idx['y'][i_x]
            except:
                x_, y_ = i_x, 0
            plt.text(x_ + wid / 2, y_+1, y_,ha='center',fontsize=10)
        
    else:
        plt.bar(np.array(idx['x']) - wid / 2, np.array(idx['y']), width=wid, label=lb + ' primer pairs', color=colors[lb], )
        for i_x in x_des_only:
            try:
                x_, y_ = idx['x'][i_x], idx['y'][i_x]
            except:
                x_, y_ = i_x, 0
            plt.text(x_ - wid / 2, y_+1, y_,ha='center',fontsize=10)

x_labs = [x.split('_')[0] for x in x[: -2]]
plt.xticks(x_val[: -2], [x.upper() for x in x_labs], fontsize=10, rotation=315)
plt.xlabel('The selection of the best V-region')
plt.ylabel('Number of environments')
# plt.title('Best primer for each environment')
plt.legend()
plt.grid(axis='y', linestyle='--')
# 设置边框不可见
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
# plt.tight_layout()
plt.savefig(f'data/Bacdive/Results/bacdive_best_primer_{rk_by}.svg', dpi=800, bbox_inches='tight')

# 2. 按照各个环境画堆叠柱状图
evi_category = 'Category 2'
Cate_index = bacdive_best_primer[evi_category].value_counts()
Cate_index = list(Cate_index[Cate_index >= 8].index)
Cate_index.remove('#Other')
# vv_index = x
vv_index = x_labs

df_category_bar = []
for cate_i, df in bacdive_best_primer.groupby(evi_category,):
    if cate_i not in Cate_index:
        continue
    vv_cnt = dict(df['VV_region'].value_counts()) # bar_label用作备选方案
    vv_cnt[evi_category] = cate_i
    df_category_bar.append(vv_cnt)

df_category_bar = pd.DataFrame(df_category_bar, columns = [evi_category] + vv_index)
df_category_bar.fillna(0, inplace=True)
df_category_bar.index = df_category_bar[evi_category]
df_category_bar.drop(columns=[evi_category], inplace=True)

# df_category_bar = df_category_bar.loc[Cate_index, :] # 按照环境数目的顺序来重新排序
df_category_bar = df_category_bar / df_category_bar.sum(axis=1).values.reshape(-1, 1)
df_category_bar['v3_v5_sum'] = df_category_bar.iloc[:, : 2].sum(axis=1) # v3v4 + v4v5去排序
df_category_bar.sort_values(by = 'v3_v5_sum', inplace=True, ascending=False)

plt.figure(figsize=(10, 20))
fig, ax = plt.subplots()

cmp = mpl.cm.get_cmap('Set3', len(vv_index))
for vv_i in range(len(vv_index)):
    plot_info = df_category_bar.iloc[:, vv_i]
    if vv_i == 0:
        plt.bar(np.arange(plot_info.shape[0]), plot_info, label=vv_index[vv_i].upper(), color= cmp(vv_i / len(vv_index)))
    else:
        plot_info_bottom = df_category_bar.iloc[:, : vv_i].sum(axis=1)
        plt.bar(np.arange(plot_info.shape[0]), plot_info, bottom=plot_info_bottom, label=vv_index[vv_i].upper(), color= cmp(vv_i / len(vv_index)))

xtick_cate_nm = list(df_category_bar.index)
plt.xticks(np.arange(plot_info.shape[0]), xtick_cate_nm, fontsize=8, rotation=315, ha='left')
# 调整刻度标签与坐标轴的距离
plt.tick_params(axis='x', pad=1)  # pad 参数设置刻度标签与坐标轴的距离

# 自定义 y 轴上的刻度标签为百分比形式
plt.ylabel('Relative ratio (%)')
def percentage_formatter(x, pos):
    return f'{x*100:.0f}%'
plt.gca().yaxis.set_major_formatter(FuncFormatter(percentage_formatter))


# plt.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
# plt.title('Best primer for each category.')
# 设置边框不可见
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

plt.savefig(f'data/Bacdive/Results/bacdive_best_primer_category_{rk_by}.svg', dpi=800, bbox_inches='tight')

In [ ]:
# 用来进行v3v4细致比较的func
def compDesign_v3v4(de_best_pri_ge_list, uni_target_pri_ge_list, ):
    # 成对样本检验p-value
    pval_paired_test = wilcoxon(de_best_pri_ge_list, uni_target_pri_ge_list, zero_method='wilcox', correction=False).pvalue
    return pval_paired_test
    
    
xlsx_write = pd.ExcelWriter(r'results/bacdive/detail.xlsx')
# 3. 每个环境做细致的genus比较
# 0521：细致比较每一个环境和v3v4的区别
uni_comp_vregion = 'v3v4'

rk_by = 'Genus_accuracy'
bacdive_root = 'IN_bacdive_middle_v2'

vv_uni_comp = univer_primer_table[uni_comp_vregion] #'338F_806R_v3v4'
accDiff_cut_off = 0.05
pval_cutoff = 0.05 # 画图的时候的pval cut off
# 0619: 进行列联表检验的参数
eff_ge_moreThan_seqNum = 10 # 只保留那些序列数大于一定数目的ge
prevalent_rare_divNum = 100 # 用来划分prevalent和rare的genus数目

ge_spectrum_comp = []
ge_paired_test = []
ge_corr_test = []
for evi in list(bacdive_best_primer['Environment']):
    evi_summary_df = pd.read_csv(os.path.join(bacdive_root, evi, 'detail_Genus_accuracy.csv'))
    evi_summary_df.index = evi_summary_df['tax_name']
    de_best_pri = list(bacdive_best_primer[bacdive_best_primer['Environment'] == evi]['Designed_best_primer'])[0]
    try:
        # 找到比较对象的ge-spectrum
        de_best_pri_ge_list = evi_summary_df[de_best_pri]
        uni_target_pri_ge_list = evi_summary_df[vv_uni_comp]
        ge_representative_seqNum = evi_summary_df['tax_num']
    except:
        ge_spectrum_comp.append(None)
        ge_paired_test.append(None)
        continue
    
    # 1.计算成对样本检验情况；2.后续加入seq num列联表检验
    pval_paired_test = compDesign_v3v4(de_best_pri_ge_list, uni_target_pri_ge_list)
    ge_paired_test.append(pval_paired_test)
    
    # 细致看哪些ge超过了cut_off
    evi_ge_spectrum = {}
    a, b, c, d, e, f = 0, 0, 0, 0, 0, 0
    
    for i in range(len(de_best_pri_ge_list)):
        # 去除那些只有较少representative序列的ge; TODO:试一下acc计算的时候，也去除
        if ge_representative_seqNum[i] < eff_ge_moreThan_seqNum:
            continue
            
        if de_best_pri_ge_list[i] - uni_target_pri_ge_list[i] > accDiff_cut_off:
            # 显著优于uni primer
            evi_ge_spectrum.update({de_best_pri_ge_list.index[i]: (de_best_pri_ge_list[i], uni_target_pri_ge_list[i])})
            if ge_representative_seqNum[i] > prevalent_rare_divNum:
                a += 1
            else:
                d += 1
                
        elif de_best_pri_ge_list[i] - uni_target_pri_ge_list[i] < -accDiff_cut_off:
            # 显著更差于uni primer
            if ge_representative_seqNum[i] > prevalent_rare_divNum:
                b += 1
            else:
                e += 1
        
        else:
            # 没有显著差异
            if ge_representative_seqNum[i] > prevalent_rare_divNum:
                c += 1
            else:
                f += 1
    try:
        stat_data = [[a, b, c],
                    [d, e, f]]
        chi2, p, dof, expected = chi2_contingency(stat_data)
    except:
        p = -1.0
        stat_data = []
    
    ge_corr_test.append((p, stat_data))
    ge_spectrum_comp.append(evi_ge_spectrum)
    
bacdive_best_primer[f'Spectrum_comp_{vv_uni_comp}'] = ge_spectrum_comp
bacdive_best_primer[f'Paired_test_{vv_uni_comp}'] = ge_paired_test
bacdive_best_primer[f'Corr_test_{vv_uni_comp}_{prevalent_rare_divNum}_{eff_ge_moreThan_seqNum}'] = ge_corr_test

# 4. 画按照genus-num递减的环境散点图
bacdive_best_primer.sort_values(by='Genus_num', inplace=True, ascending=False)
bacdive_best_primer.reset_index(inplace=True, drop=True)
# plt.style.use('ggplot')
plt.figure()
# bacdive_best_primer.iloc[:180, :].plot.scatter(x='Genus_num', y='Value', figsize=(8, 6), s=50, c='DarkBlue', )
plt.plot(np.log(bacdive_best_primer['Genus_num']), bacdive_best_primer['Value'], 'o', markersize=3)
plt.title('Best primer of environment with different genus number')
plt.savefig(f'results/bacdive/bacdive_best_primer_genusNum.pdf', dpi=300)

# 细致看看比v3v4更好的环境
bacdive_best_primer[f'Design_ninus_{uni_comp_vregion}'] = bacdive_best_primer['Designed_best_val'] - bacdive_best_primer[vv_uni_comp]

print('Degree of designed primer better than universal primer: \n', bacdive_best_primer[f'Designed_best_val'].describe(), bacdive_best_primer[vv_uni_comp].describe()) # bacdive_best_primer[bacdive_best_primer[vv_uni_comp] > 1e-5][f'Design_ninus_{uni_comp_vregion}'].describe()
bacdive_best_primer_greaterThan_v3v4 = bacdive_best_primer[(bacdive_best_primer[f'Design_ninus_{uni_comp_vregion}'] >= accDiff_cut_off) & (bacdive_best_primer[vv_uni_comp] > 1e-5) & (bacdive_best_primer[f'Paired_test_{vv_uni_comp}'] < pval_cutoff)]
bacdive_best_primer_greaterThan_v3v4.reset_index(inplace=True, drop=True)
bacdive_best_primer_greaterThan_v3v4.to_excel(xlsx_write, sheet_name=f'Greater_than_{uni_comp_vregion}_{accDiff_cut_off}_{pval_cutoff}', index=False)
print('Degree of designed primer better than universal primer: \n', bacdive_best_primer_greaterThan_v3v4['Design_ninus_v3v4'].describe())

# 和v3v4比较的时候画一个成对样本检验p-value和acc大于的火山图
df = bacdive_best_primer.copy()
df = df[df[vv_uni_comp] > 1e-5] # 排除掉没有v3v4结果的
df.reset_index(inplace=True, drop=True)
df[f'Paired_test_{vv_uni_comp}']=df[f'Paired_test_{vv_uni_comp}'].apply(lambda x:-np.log10(x+1e-14))

plt.figure(figsize=(12,10))
plt.scatter(x=df[f'Design_ninus_{uni_comp_vregion}'],y=df[f'Paired_test_{vv_uni_comp}'],s=10,label="Not significant",color="grey")
# highlight down- or up- regulated genes
down = df[(df[f'Design_ninus_{uni_comp_vregion}']<=-accDiff_cut_off) & (df[f'Paired_test_{vv_uni_comp}']>=-math.log10(pval_cutoff))]
up = df[(df[f'Design_ninus_{uni_comp_vregion}']>=accDiff_cut_off) & (df[f'Paired_test_{vv_uni_comp}']>=-math.log10(pval_cutoff))]

plt.scatter(x=down[f'Design_ninus_{uni_comp_vregion}'],y=down[f'Paired_test_{vv_uni_comp}'],s=15,label="16sDeepEcoPrimer is worse",color="blue")
plt.scatter(x=up[f'Design_ninus_{uni_comp_vregion}'],y=up[f'Paired_test_{vv_uni_comp}'],s=15,label="16sDeepEcoPrimer is better",color="red")

texts=[]
for i,r in down.iterrows():
    texts.append(plt.text(x=r[f'Design_ninus_{uni_comp_vregion}'],y=r[f'Paired_test_{vv_uni_comp}'],s=str(r['Environment']))) 
for i,r in up.iterrows():
    texts.append(plt.text(x=r[f'Design_ninus_{uni_comp_vregion}'],y=r[f'Paired_test_{vv_uni_comp}'],s=str(r['Environment']))) 

if len(texts) > 0:
    adjust_text(texts, arrowprops=dict(arrowstyle='-', color='black'))


plt.xlabel(f"Classification accuracy of designed primer minus that of 338F_806R", fontsize=15)
plt.ylabel("-logFDR", fontsize=15)
# 自定义 x轴上的刻度标签为百分比形式
def percentage_formatter(x, pos):
    return f'{x*100:.0f}%'
plt.gca().xaxis.set_major_formatter(FuncFormatter(percentage_formatter))

# plt.title(f'Difference of classification accuracy between designed primer and 338F_806R') #  {vv_uni_comp}
plt.axvline(-accDiff_cut_off,color="grey",linestyle="--")
plt.axvline(accDiff_cut_off,color="grey",linestyle="--")
plt.axhline(-math.log10(pval_cutoff),color="grey",linestyle="--")

plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.legend(fontsize='large')
plt.savefig(f'results/bacdive/bacdive_compWith_{vv_uni_comp}.svg', dpi=600, bbox_inches='tight')


# 5. 看看当最优是universal的时候，设计的效果如何
bacdive_best_primer['Uni_ninus_designed'] = bacdive_best_primer['Designed_best_val'] - bacdive_best_primer['Value']
bacdive_best_primer_uni_better = bacdive_best_primer[bacdive_best_primer['Pri_type'] == 'universal']
print('Info of environment when universal primer is better than designed primer: \n', bacdive_best_primer_uni_better['Uni_ninus_designed'].describe())
print(bacdive_best_primer_uni_better['Designed_best_val'].describe(), bacdive_best_primer_uni_better['Universal_best_val'].describe())

plt.figure()

# 6. 看看当最优是design的时候，能够优秀多少
bacdive_best_primer_de_better = bacdive_best_primer[bacdive_best_primer['Pri_type'] == 'designed']
print('Info of environment when designed primer is better: \n', bacdive_best_primer_de_better['De_minus_bestUni'].describe())
print(bacdive_best_primer_de_better['Designed_best_val'].describe(), bacdive_best_primer_de_better['Universal_best_val'].describe())

ind_plot = ['Designed_best_val', 'Value']
for ind in ind_plot:
    plt.plot(bacdive_best_primer_uni_better[ind], label=ind)
plt.legend()

xlsx_write.close()

## primer best df中新加入了一些指标，重新保存
bacdive_best_primer.to_csv(f'results/bacdive/bacdive_best_primer_{rk_by}_bigEvi_{vv_uni_comp}_Comp.csv', index=False)

bacdive_best_primer_uni_better.shape

In [ ]:
# 7. 看看不同categories的
comp_pris = ['27F_338R_v1v2', '27F_533R_v1v3', '515F_907R_v4v5', '338F_806R_v3v4', '515F_806R_v4v4', '969F_1406R_v6v8', 'Designed_best_val']
evi_category = 'Category 2'
Cate_index = bacdive_best_primer[evi_category].value_counts()
Cate_index = list(Cate_index[Cate_index >= 8].index)
Cate_index.remove('#Other')

vv_index = x
vv_index.sort()

df_category_std = {
    'Group':[],
    'Subgroup':[],
    'Value':[]
    }
for cate_i, df in bacdive_best_primer.groupby(evi_category,):
    if cate_i not in Cate_index:
        continue
    gp1 = df['De_minus_bestUni'].values
    gp2 = df[comp_pris].std(axis=1).values
    gp1 = []
    
    df_category_std['Group'] += [cate_i] * (len(gp1) + len(gp2))
    df_category_std['Subgroup'] += ['diff_1'] * len(gp1) + ['diff_2'] * len(gp2)
    df_category_std['Value'] += list(gp1) + list(gp2)
df_category_std = pd.DataFrame(df_category_std)

# 计算每个组别的均值并排序
mean_by_group = df_category_std.groupby('Group')['Value'].mean().sort_values()
# 根据均值排序后的组别顺序，对数据框进行排序
data_sorted = df_category_std.sort_values(by='Group', key=lambda x: x.map(mean_by_group))

# 设置顶级科学期刊配色方案
color_palette = sns.color_palette("husl", 3)

# 设置绘图风格为 'ticks'
sns.set_style("ticks")

# 绘制双列的箱线图
plt.figure(figsize=(12, 9))  # 设置图形大小
sns.boxplot(x='Group', y='Value', data=data_sorted, width=0.45, color=color_palette[2])  # 绘制箱线图

# 添加标题和坐标轴标签
# plt.title('Primer bias in different categories')
plt.ylabel('Primer bias (%)', fontsize=20)
plt.xlabel('')
plt.xticks(fontsize=20, rotation=315, ha='left')
plt.yticks(fontsize=20)
# 调整刻度标签与坐标轴的距离
plt.tick_params(axis='x', pad=1)  # pad 参数设置刻度标签与坐标轴的距离
plt.grid(axis='y', linestyle='--')

# 自定义 y 轴上的刻度标签为百分比形式
def percentage_formatter(x, pos):
    return f'{x*100:.0f}%'
plt.gca().yaxis.set_major_formatter(FuncFormatter(percentage_formatter))

# 去除顶部和右侧的边框线
sns.despine()

plt.savefig(f'data/Bacdive/Results/Primer_bias.svg', dpi=600, bbox_inches='tight')


In [ ]:
# 8. 看一下各个环境设计出的引物具体target位点，尝试解释为何我们的引物更好

## 0630：找各个primer binding site的序列
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO

bacdive_best_primer_metadf = pd.read_csv('results/bacdive/bacdive_best_primer_Genus_accuracy_bigEvi_338F_806R_v3v4_Comp.csv')
bac_pri_root = 'bacdiveEvi_primers'

def find_opt_pri_seq(evi_df):
    opt_primer_infos = []
    for evi_nm, pri_nm in zip(list(evi_df['Environment']), list(evi_df['Designed_best_primer'])):
        # 首先提取meta信息
        pri_nm += 'r'
        vv, fr = pri_nm.split('_')[-2], pri_nm.split('_')[-1]
        evi_root = os.path.join(bac_pri_root, evi_nm, vv)
        f_num, r_num = int(fr.strip('r').split('f')[0]), int(fr.strip('r').split('f')[1])
        
        # 从csv文件中找到primer的详细信息
        f_pri_df, r_pri_df = pd.read_csv(os.path.join(evi_root, f'forward_primer.csv')), pd.read_csv(os.path.join(evi_root, f'reverse_primer.csv'))
        f_pri_df.sort_values(by='cov_num_k_3', ascending=False, inplace=True)
        f_pri_df.reset_index(inplace=True, drop = True)
        r_pri_df.sort_values(by='cov_num_k_3', ascending=False, inplace=True)
        r_pri_df.reset_index(inplace=True, drop = True)

        # 整合optimal primer信息
        opt_primer_info_head = ['primer_atgc', 'primer_degenum', 'GC_ratio', 'Tm']
        opt_primer_info_tab = {}
        for opt_pri_if in opt_primer_info_head:
            f_pri, r_pri = f_pri_df.loc[f_num, opt_pri_if], r_pri_df.loc[r_num, opt_pri_if]
            opt_primer_info_tab[opt_pri_if + '_f'] = f_pri
            opt_primer_info_tab[opt_pri_if + '_r'] = r_pri

        opt_primer_infos.append(opt_primer_info_tab)
    evi_df['opt_primer_info'] = opt_primer_infos
    return evi_df

bacdive_best_primer_metadf = find_opt_pri_seq(bacdive_best_primer_metadf)
bacdive_best_primer_metadf.to_csv('results/bacdive/bacdive_optPrimerInfo.csv', index=False) # 保存来看看

# 找到v3v4最优的那些设计引物
design_pri_targetVregion = 'v3v4'
bacdive_designed_primer_thisVregion = bacdive_best_primer_metadf[(bacdive_best_primer_metadf['Designed_best_vv_region'] == design_pri_targetVregion) & (bacdive_best_primer_metadf['Pri_type'] == 'designed')]
bacdive_designed_primer_thisVregion = pd.DataFrame(bacdive_designed_primer_thisVregion, columns=['Designed_best_primer', 'Environment', 'opt_primer_info'])
bacdive_designed_primer_thisVregion.reset_index(inplace=True, drop=True)

univer_primer_table_ori = pd.read_excel('ref_database/universal_primers/primers_sequencing_bacdiveComp.xlsx')# 进行细致比较的时候会用到
univer_primer_table_ori = univer_primer_table_ori[univer_primer_table_ori['v_region'] == design_pri_targetVregion].reset_index(drop=True)
uni_pri_atgcs = {'primer_atgc_f':univer_primer_table_ori['forward_seq'][0].strip("5'- ").strip("-3'"), 'primer_atgc_r':univer_primer_table_ori['reverse_seq'][0].strip("5'- ").strip("-3'")}


# 开始比对到ecoli上
# 1. 先建立数据库
ecoli_ref = 'ref_database/ecoli_K12_MG1655.fna'
db = ecoli_ref.split(".")[0]
cmd_mkDatabase = f'makeblastdb -in {ecoli_ref} -dbtype nucl -out {db}'
os.system(cmd_mkDatabase)

def blast_cmd(query_primer="", db="", out_path = ''):
    cmd = f"blastn -db {db} -query "
    cmd += query_primer
    cmd += " -outfmt 7 "
    cmd += "-out "
    cmd += out_path
    cmd += " -task blastn-short -word_size 4 -evalue 1 -max_target_seqs 1000000"
    
    os.system(cmd)
    
def parse_blastTxt(blast_txt = '/data3/hyzhang/ont/16s_RNA_seg/res/blast_res/query.txt'):
    with open(blast_txt, 'r') as f:
        lines = f.readlines()
    index_line = lines[3]
    index_line = index_line.split(': ')[-1]
    index_line = index_line.split(',')
    index_line = [x.strip(' ') for x in index_line]
    index_line = ['_'.join(x.split()) for x in index_line]

    lines = [x.strip('\n').split('\t') for x in lines if not x.startswith('#')]
    
    blast_df = pd.DataFrame(lines)
    blast_df.columns = index_line
    return blast_df

# 2. 每个环境比对
temp_dir = 'temp_align'
df_evi_primer_ecoli = []
for evi_i in range(bacdive_designed_primer_thisVregion.shape[0]):
    evi_nm = bacdive_designed_primer_thisVregion.loc[evi_i, 'Environment']
    f_pri, r_pri = bacdive_designed_primer_thisVregion.loc[evi_i, 'opt_primer_info']['primer_atgc_f'], bacdive_designed_primer_thisVregion.loc[evi_i, 'opt_primer_info']['primer_atgc_r']
    records = [
        SeqRecord(seq=Seq(f_pri), id='forward', description= ''),
        SeqRecord(seq=Seq(r_pri), id='reverse', description= '')
    ]

    with open(f'./{temp_dir}/temp_al.fna', 'w') as f:
        SeqIO.write(records, f, 'fasta')
        
    blast_cmd(query_primer=f'./{temp_dir}/temp_al.fna', db=db, out_path=f"./{temp_dir}/{evi_nm}.fna")
    blast_df = parse_blastTxt(blast_txt=f"./{temp_dir}/{evi_nm}.fna")
    
    info_t = {'Environment': evi_nm}
    for pri_ty, pri_atgc in zip(['forward', 'reverse'], [f_pri, r_pri]):
        info_t[pri_ty + '_seq'] = pri_atgc
        df_ = blast_df[blast_df['query_acc.ver'] == pri_ty].reset_index(drop=True)
        df_.sort_values(by='evalue', inplace=True, ascending=True)
        info_t[pri_ty + '_bitscore'] = df_.loc[0, 'bit_score']
        if pri_ty == 'forward':
            info_t[pri_ty + '_start'] = int(df_.loc[0, 's._start']) - (int(df_.loc[0, 'q._start']) - 1)
        else:
            info_t[pri_ty + '_start'] = int(df_.loc[0, 's._start']) + (int(df_.loc[0, 'q._start']) - 1)
    df_evi_primer_ecoli.append(info_t)

# 3. uni引物结果
evi_nm = f'universal_{design_pri_targetVregion}'
f_pri, r_pri = uni_pri_atgcs['primer_atgc_f'], uni_pri_atgcs['primer_atgc_r']
records = [
    SeqRecord(seq=Seq(f_pri), id='forward', description= ''),
    SeqRecord(seq=Seq(r_pri), id='reverse', description= '')
]

with open(f'./{temp_dir}/temp_al.fna', 'w') as f:
    SeqIO.write(records, f, 'fasta')
    
blast_cmd(query_primer=f'./{temp_dir}/temp_al.fna', db=db, out_path=f"./{temp_dir}/{evi_nm}.fna")
blast_df = parse_blastTxt(blast_txt=f"./{temp_dir}/{evi_nm}.fna")

info_t = {'Environment': evi_nm}
for pri_ty, pri_atgc in zip(['forward', 'reverse'], [f_pri, r_pri]):
    info_t[pri_ty + '_seq'] = pri_atgc
    df_ = blast_df[blast_df['query_acc.ver'] == pri_ty].reset_index(drop=True)
    df_.sort_values(by='evalue', inplace=True, ascending=True)
    info_t[pri_ty + '_bitscore'] = df_.loc[0, 'bit_score']
    if pri_ty == 'forward':
            info_t[pri_ty + '_start'] = int(df_.loc[0, 's._start']) - (int(df_.loc[0, 'q._start']) - 1)
    else:
        info_t[pri_ty + '_start'] = int(df_.loc[0, 's._start']) + (int(df_.loc[0, 'q._start']) - 1)
df_evi_primer_ecoli.append(info_t)


df_evi_primer_ecoli = pd.DataFrame(df_evi_primer_ecoli)
df_evi_primer_ecoli.to_csv(f'results/bacdive/evi_opt_pri_alignEcoli_{design_pri_targetVregion}.csv', index=False)


In [ ]:
df_evi_primer_ecoli['forward_start'].hist(bins = 50)

In [ ]:
df_evi_primer_ecoli['reverse_start'].hist(bins = 110)
plt.xlim([780, 830])

#### 4. 构建用于EcoPrimer测试的demo输入数据


In [5]:
demo_input_ori = pd.read_csv('input/demo_input_EcoPrimer_ori.csv')
demo_root = 'input/metagenomic_input'

for demo_i in range(demo_input_ori.shape[0]):
    ori_path_single_end = demo_input_ori.iloc[demo_i, 2]
    ori_path_pair_end = demo_input_ori.iloc[demo_i, 1]
    srr_id = demo_input_ori.iloc[demo_i, 0]
    srr_id_ori = os.path.basename(ori_path_single_end).split('_SE1.pick.fasta')[0]
    
    demo_single_end = os.path.join(demo_root, srr_id + '_SE1.pick.fasta')
    demo_pair_end = os.path.join(demo_root, srr_id + '_PEmerged.pick.fasta')
    print(f'head -n 2000 {ori_path_single_end} > {demo_single_end}')
    os.system(f'head -n 2000 {ori_path_single_end} > {demo_single_end}')
    os.system(f'head -n 2000 {ori_path_pair_end} > {demo_pair_end}')
    
    demo_input_ori.iloc[demo_i, 1] = demo_pair_end
    demo_input_ori.iloc[demo_i, 2] = demo_single_end
    demo_input_ori.iloc[demo_i, 3] = srr_id_ori

demo_input_ori.to_csv('input/demo_input_EcoPrimer.csv', index=False)



head -n 2000 /data3/hyzhang/16sDeepSeg_primer_2023/mipe_metagenome_primer_screen/16s_metagenomes_data/sputum_gq0505/metagenome/clean_ssu_seqs/zhuyoubao20190508_FDSW202112160_1r_SE1.pick.fasta > input/metagenomic_input/demo1_SE1.pick.fasta
head -n 2000 /data3/hyzhang/16sDeepSeg_primer_2023/mipe_metagenome_primer_screen/16s_metagenomes_data/sputum_gq0505/metagenome/clean_ssu_seqs/zhaozhangxiang20190508_FDSW202112182_1r_SE1.pick.fasta > input/metagenomic_input/demo2_SE1.pick.fasta


In [ ]:
demo_input_ori = pd.read_csv('input/demo_input_EcoPrimer_ori.csv')
demo_root = 'input/metagenomic_input'

for demo_i in range(demo_input_ori.shape[0]):
    ori_path_single_end = demo_input_ori.iloc[demo_i, 2]
    ori_path_pair_end = demo_input_ori.iloc[demo_i, 1]
    srr_id = demo_input_ori.iloc[demo_i, 0]
    srr_id_ori = os.path.basename(ori_path_single_end).split('_SE1.pick.fasta')[0]
    
    demo_single_end = os.path.join(demo_root, srr_id + '_SE1.pick.fasta')
    demo_pair_end = os.path.join(demo_root, srr_id + '_PEmerged.pick.fasta')
    print(f'head -n 2000 {ori_path_single_end} > {demo_single_end}')
    os.system(f'head -n 2000 {ori_path_single_end} > {demo_single_end}')
    os.system(f'head -n 2000 {ori_path_pair_end} > {demo_pair_end}')
